In [2]:
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import json
sns.set_style('darkgrid')
%matplotlib inline


In [4]:
train = pd.read_csv('train.csv', index_col='row_id')
# train.drop_duplicates(keep='first', inplace=True)
train.reset_index(drop=True, inplace=True)
test = pd.read_csv('test.csv', index_col='row_id')
# data = pd.concat([train, test], sort=False)
# train = data[:len(train)]
# test = data[len(train):]
sample_submission = pd.read_csv(
    'sample_submission.csv')

In [5]:
test

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
row_id,,,,,,,,,,,,,,,,,,,,,
200000,-9.536743e-07,-0.000002,-9.153442e-07,0.000024,0.000034,-0.000002,0.000021,0.000024,-0.000009,-0.000008,...,0.000039,0.000085,0.000055,0.000108,0.000090,0.000059,0.000010,0.000006,0.000027,4.632568e-08
200001,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,0.001800,-0.000240,0.001800,-0.000114,0.000957,-0.000010,...,-0.000043,0.000914,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07
200002,4.632568e-08,0.000003,8.465576e-08,-0.000014,0.000007,-0.000005,-0.000004,0.000003,0.000004,-0.000008,...,0.000041,0.000102,0.000084,0.000111,0.000117,0.000070,0.000020,0.000030,0.000021,4.632568e-08
200003,-9.536743e-07,-0.000008,8.084656e-06,0.000216,0.000420,0.000514,0.000452,0.000187,-0.000005,-0.000008,...,0.000069,0.000158,0.000098,0.000175,0.000217,0.000150,0.000018,0.000016,0.000051,-9.536743e-07
200004,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,-0.000086,0.000914,-0.000043,-0.000010,-0.000010,0.000990,-9.536743e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-9.536743e-07,-0.000008,-4.291534e-05,-0.000108,-0.000194,-0.000234,-0.000195,-0.000110,-0.000043,-0.000010,...,0.000159,0.000559,0.000489,0.000499,0.000899,0.000443,0.000047,0.000148,0.000101,4.046326e-06
299996,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07
299997,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,0.000914,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07


In [13]:
data = pd.concat([train, test], sort=False)
data_duplicated = data[data.duplicated(subset=test.columns, keep=False)]
data_duplicated

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Streptococcus_pyogenes
1,-9.536743e-07,-0.00001,-0.000043,0.000886,-0.0002,0.00076,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Salmonella_enterica
4,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Enterococcus_hirae
6,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Streptococcus_pyogenes
8,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,0.000957,-0.00001,-0.00001,-0.00001,-9.536743e-07,Streptococcus_pneumoniae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299992,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN
299993,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,0.00976,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN
299996,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN
299997,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,0.000914,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN


In [14]:
data_duplicated.iloc[3:, :]

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
6,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Streptococcus_pyogenes
8,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,0.000957,-0.00001,-0.00001,-0.00001,-9.536743e-07,Streptococcus_pneumoniae
10,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,0.000957,-0.00001,0.00099,-0.00001,-9.536743e-07,Streptococcus_pneumoniae
12,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Escherichia_fergusonii
14,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Enterococcus_hirae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299992,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN
299993,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,0.00976,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN
299996,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN
299997,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,...,-0.000086,-0.000043,0.000914,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,NaN


In [23]:
save_dict = {'index': [], 'target':[]}
for i, row in test.iterrows():
    statement = ''
    for c in test.columns:
        statement += " {}=={} & ".format(c, row[c])
    statement += "1==1"
    q = train.query(statement)
    if not q.empty:
        save_dict['index'].append(i)
        save_dict['target'].append(q['target'].iloc[-1])
        print(save_dict)


KeyError: 0

In [ ]:
with open('duplicate.json', 'w') as f:
    json.dump(save_dict, f)

In [29]:
q['target'].iloc[-1]

'Streptococcus_pneumoniae'